##### Copyright 2023 The MediaPipe Authors. All Rights Reserved.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Hand Landmarks Detection with MediaPipe Tasks

This notebook shows you how to use MediaPipe Tasks Python API to detect hand landmarks from images.

## Preparation

Let's start with installing MediaPipe.

Then download an off-the-shelf model bundle. Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/hand_landmarker#models) for more information about this model bundle.

## Visualization utilities

In [6]:
#@markdown We implemented some functions to visualize the hand landmark detection results. <br/> Run the following cell to activate the functions.
import cv2 as cv
from cmath import sqrt
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import cv2

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green
def manos_centro(puntos,numero):
  return sqrt(((puntos[numero].x)-(puntos[0].x))**2+((puntos[numero].y)-(puntos[0].y))**2+((puntos[numero].y)-(puntos[0].y))**2).real

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # hand_landmarks_list[idx][]
    puntos=[]
    for landmark in hand_landmarks:
      puntos.append(landmark)

    if manos_centro(puntos,8)<manos_centro(puntos,7) and manos_centro(puntos,12)<manos_centro(puntos,11) and manos_centro(puntos,16)<manos_centro(puntos,15) and manos_centro(puntos,20)<manos_centro(puntos,19):
        cv.putText(annotated_image, 'piedra',
                  (text_x, text_y), cv.FONT_HERSHEY_DUPLEX,
                  FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv.LINE_AA)
    elif manos_centro(puntos,8)>manos_centro(puntos,7) and manos_centro(puntos,12)>manos_centro(puntos,11) and manos_centro(puntos,16)>manos_centro(puntos,15) and manos_centro(puntos,20)>manos_centro(puntos,19):
        cv.putText(annotated_image, 'papel',
                  (text_x, text_y), cv.FONT_HERSHEY_DUPLEX,
                  FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv.LINE_AA)   
    elif manos_centro(puntos,8)>manos_centro(puntos,7) and manos_centro(puntos,12)>manos_centro(puntos,11) and manos_centro(puntos,16)<manos_centro(puntos,15) and manos_centro(puntos,20)<manos_centro(puntos,19):
        cv.putText(annotated_image, 'tijera',
                  (text_x, text_y), cv.FONT_HERSHEY_DUPLEX,
                  FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv.LINE_AA)   
    else: 
        cv.putText(annotated_image, 'nada',
                  (text_x, text_y), cv.FONT_HERSHEY_DUPLEX,
                  FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv.LINE_AA)   

  return annotated_image

## Download test image

Let's grab a test image that we'll use later. The image is from [Unsplash](https://unsplash.com/photos/mt2fyrdXxzk).

Optionally, you can upload your own image. If you want to do so, uncomment and run the cell below.

In [ ]:
# from google.colab import files
# uploaded = files.upload()

# for filename in uploaded:
#   content = uploaded[filename]
#   with open(filename, 'wb') as f:
#     f.write(content)

# if len(uploaded.keys()):
#   IMAGE_FILE = next(iter(uploaded))
#   print('Uploaded file:', IMAGE_FILE)

## Running inference and visualizing the results

Here are the steps to run hand landmark detection using MediaPipe.

Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/hand_landmarker/python) to learn more about configuration options that this solution supports.


In [7]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import time

# Paso 1: Crear un objeto HandLandmarker.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options, running_mode=vision.RunningMode.VIDEO, num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

# Paso 2: Iniciar la captura de video desde la cámara.
cap = cv2.VideoCapture(0)  # 0 para la cámara predeterminada
timestamp = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    image = frame
    
    # Paso 3: Convertir la imagen al formato RGB de MediaPipe.
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb)
    
    # Paso 4: Detectar landmarks de manos.
    f_timestamp = round((time.time() - timestamp) * 1000)
    hand_landmarker_result = detector.detect_for_video(mp_image, f_timestamp)
    
    # Paso 5: Dibujar los landmarks en la imagen.
    annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), hand_landmarker_result)
    
    # Paso 6: Mostrar la imagen corregida.
    cv2.imshow('Hand Landmarks Detection', annotated_image[:, :, ::-1])  # Convertir de RGB a BGR para mostrar con OpenCV

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Paso 7: Liberar recursos y cerrar ventanas.
cap.release()
cv2.destroyAllWindows()




